## Import the needed Library

This dashboard use **Cov19API** to access data from PHE website  
use **json** library to convert and store data from PHE  
use **pandas** and **matplotlib** library to visualise the data, the data would be saved as new_data.pkl file  
use **IPython** to make interactive widgets and refresh function

In [1]:
#Libraries
from uk_covid19 import Cov19API
import json
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Define the data_access() function
I define the data_access() function to access data from PHE website.  
While calling this function, it will initialize the api and get data automatically.The data contains date, total number of people who take the 1,2,3 dose of vaccination and the full vaccination rate.
All data is saved in new_data.json file.  

### data strcture of data.json

- mydata[dict]
    - data[list]
        - entry1[dict]
            - date[str]
            - dose1[str]
            - dose2[str]
            - dose3[str]
            - FullVacRate[str]
        - entry2
        - ......
    - lastUpdate
    - length
    - totalPages


In [2]:
def data_access():
    '''access data from PHE with defined api, save as "new_data.json" file '''
    #define the data structure
    filters = [
    'areaType=region',
    'areaName=London'
    ]
    #metric name cannot start with number, like 1stdose etc.#
    structure = {
        'date':'date',
        'dose1':'cumPeopleVaccinatedFirstDoseByVaccinationDate',
        'dose2':'cumPeopleVaccinatedSecondDoseByVaccinationDate',
        'dose3':'cumPeopleVaccinatedThirdInjectionByVaccinationDate',
        'FullVacRate':'cumVaccinationCompleteCoverageByVaccinationDatePercentage'
    }
    api = Cov19API(filters=filters, structure=structure)
    
    #get data from PHE
    vac = api.get_json()
    #dump the data from PHE
    with open('new_data.json', 'wt') as OUTF:
        json.dump(vac, OUTF)

## Define the data_wrangling() function
This function is used to extract data from .json file created by data_access() function, wangling it into a pandas dataframe, and save as .pkl file


In [3]:
def data_wrangling(fname):
    '''parse the .json file, wangling data and import them into pandas dataframe,
        save as"new_data.pkl" file '''
    
    with open(fname, 'rt') as INFILE:
        mydata = json.load(INFILE)
    
    #extract all the date from fname
    date_list = []
    for dic in mydata['data']:
        date_list.append(dic['date'])
    date_list.sort()

    #extract the first_day & last_day
    #note: the order of key-value pairs in dictionary may be chaos, need to sort first
    start_date = pd.to_datetime(date_list[0], format = '%Y-%m-%d')
    last_date = pd.to_datetime(date_list[-1], format = '%Y-%m-%d')    
    
    #create a new data frame with given collumn
    index = pd.date_range(start_date, last_date, freq='D')
    mydata_df = pd.DataFrame(index=index, columns=['dose1', 'dose2', 'dose3', 'FullVacRate'])
#    mydata_df.plot()
    
    #put values into the dataframe
    for entry in mydata['data']:
        date = pd.to_datetime(entry['date'], format = '%Y-%m-%d') #use date as index
        for col in ['dose1', 'dose2', 'dose3', 'FullVacRate']: 
            if pd.isna(mydata_df.loc[date, col]): #check the position if it is None, otherwise it's duplicate
                if entry[col] == None: #if the value in json is None, replace it with 0.0
                    value = 0.0
                else:
                    value = entry[col] #if not None, use it value
                mydata_df.loc[date, col] = value
    mydata_df.to_pickle("data_df.pkl")
    return mydata_df

## Define a button
While click this button, call functions in sequence:  
data_access()  
data_wrangling(), note data_wrangling() returns a dataframe, assign it to global variable data_df  
refresh() #see definition below  

Finally change the attribute of the button.


In [4]:
def access_api(button):
    
    #download new data from PHE, save as new_data.json
    data_access()

    #wrangling the data, return a dataframe, pass the dataframe to global parameter data_df
    global data_df
    data_df = data_wrangling('new_data.json') 
    
    refresh()#change vaccols.value, trigger to redraw the plot
    
    apibutton.description = 'Newdata download'
    apibutton.icon="check"
    apibutton.button_style = 'success'
    apibutton.tooltip = 'Successful download data from PHE'
    apibutton.disabled = False #disable the button after once click it

apibutton=wdg.Button(
    description='Download data',
    disabled=False,
    button_style='danger', 
    tooltip='Click to download current Public Health England data',
    icon='download' 
)

## Define vac_graph() and create vaccols instance

vac_graph(graphcolumns) plot graph from data_df, use variable graphcolumns as y-axis  
vaccols enables interactive functionality, change vaccols.value will automatically redraw the picture

In [5]:
def vac_graph(graphcolumns):
    if len(graphcolumns) > 0:
        data_df.plot(logy = True, y=list(graphcolumns))
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

vaccols = wdg.SelectMultiple(
    options = ['dose1', 'dose2', 'dose3', 'FullVacRate'], #total options
    value = ['dose1', 'dose2', 'dose3', 'FullVacRate'],  #the colums actually plot, set default as all
    rows = 4,
    descrption = 'Injection Doses',
    disabled = False #make wediget reusable
)

## Define refresh() function
**!!!!!!!IMPORTANT!!!!!!**  
refresh() function refresh the plot by change the value of the interactive widget, which simulate the mouse click action on the widget.   

the function record the current vaccols.value, and set it to a empty list.
But if current vaccols.value is empty list, set it to vaccols.options

by doing this, vaccols value changes and then changes back. And the plot refresh twice.

In [6]:
def refresh():
    current = vaccols.value
    
    if current == []:
        ohter = vaccols.options
    else:
        other = []
    vaccols.value = other
    vaccols.value = current

## Plot the picture with interactive widget and the button

In the first run, the dashboard use old_data_df.pkl file to plot picture. 
  
And after click the Data Download below, the button call back data_access(), data_wrangling(), and refresh() functions.  
data_access() and data_wrangling() function download new data from PHE, save as new_data.json and new_data_df.pkl  
refresh() redraw the plot
  
**Notice:** the old_data_df.pkl only contains the data between 2021, after click Download Data button, new plot contains data in 2022.


In [7]:
data_df = data_wrangling('old_data.json') #initialize the data_df with old_data.json
output = wdg.interactive_output(vac_graph, {'graphcolumns': vaccols})
display(vaccols, output)

apibutton.on_click(access_api) #register the callback function with apibutton
display(apibutton) #display the widgets

SelectMultiple(index=(0, 1, 2, 3), options=('dose1', 'dose2', 'dose3', 'FullVacRate'), rows=4, value=('dose1',…

Output()

Button(button_style='danger', description='Download data', icon='download', style=ButtonStyle(), tooltip='Clic…